### **25 - Nulos y ceros en una columna**

In [2]:
import pandas as pd
import numpy as np
import os

#### **`Función para calcular:`**

- **Tipo de dato**
- **Cantidad de nulos por columna**
- **Porcentaje de nulos por columna**
- **Cantidad de 0 (numericos)**
- **Porcentaje de ceros (numericos)**

In [66]:
def calidad_datos(data):
    tipos = pd.DataFrame({'tipo': data.dtypes}, index=data.columns)
    na = pd.DataFrame({'nulos': data.isna().sum()}, index=data.columns)
    na_prop = pd.DataFrame({'porc_nulos': data.isna().sum()/data.shape[0]}, index=data.columns) # data.shape : (144, n° columnas) --> data.shape[0] : 144
    # Ejemplo: df.loc[df['year']==0].shape[0] --> Accede a los registros donde la columna 'year' tenga valor igual a 0 y luego con shape[0] nos devuelve la cantidad de esos valores,
    # dado que shape[0] nos regresa el primer valor de la tupla de dimension de esta serie, que corresponde al numero de filas 
    ceros = pd.DataFrame({'ceros':[data.loc[data[col]==0].shape[0] for col in data.columns]}, index=data.columns)
    ceros_prop = pd.DataFrame({'porc_ceros':[data.loc[data[col]==0].shape[0]/data.shape[0] for col in data.columns]}, index=data.columns)
    summary = data.describe(include='all').T

    summary['limit_inf'] = summary['mean'] - summary['std'] * 1.5
    summary['limit_sup'] = summary['mean'] + summary['std'] * 1.5

    summary['outliers'] = data.apply(lambda x: sum(np.where((x < summary['limit_inf'][x.name]) | (x > summary['limit_sup'][x.name]), 1, 0)) if x.name in summary['limit_inf'].dropna().index else 0)

    return pd.concat([tipos, na, na_prop, ceros, ceros_prop, summary], axis=1).sort_values('tipo')

calidad_datos(df)

,tipo,nulos,porc_nulos,ceros,porc_ceros,count,unique,top,freq,mean,std,min,25%,50%,75%,max,limit_inf,limit_sup,outliers
year,int64,0,0.0,0,0.0,144.0,NaN,NaN,NaN,1954.5,3.464102,1949.0,1951.75,1954.5,1957.25,1960.0,1949.303848,1959.696152,24
passengers,int64,0,0.0,0,0.0,144.0,NaN,NaN,NaN,280.298611,119.966317,104.0,180.0,265.5,360.5,622.0,100.349136,460.248087,15
month,object,0,0.0,0,0.0,144,12,January,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


#### **`>` info()**

Me devuelve información de las columnas, como también el tipo de datos de ellas

In [ ]:
df = pd.read_csv('flights.csv')
df

,year,month,passengers
0,1949,January,112
1,1949,February,118
2,1949,March,132
3,1949,April,129
4,1949,May,121
...,...,...,...
139,1960,August,606
140,1960,September,508
141,1960,October,461
142,1960,November,390


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   year        144 non-null    int64 
 1   month       144 non-null    object
 2   passengers  144 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 3.5+ KB


In [11]:
df['passengers'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 144 entries, 0 to 143
Series name: passengers
Non-Null Count  Dtype
--------------  -----
144 non-null    int64
dtypes: int64(1)
memory usage: 1.2 KB


#### **`>` isnull() o isna( )** (Es lo mismo)

- Detecta valores ausentes ==> **`NaN`**, **`NaT`**, **`None`**
- Un string vacio **`''`** no lo lee

##### Ejemplo 1

In [31]:
# Podemos utilizar estos 3 valores para generar ausencia de valor en Dataframes
x = (pd.NaT, np.nan, None)
[pd.isna(i) for i in x]

[True, True, True]

In [80]:
de = pd.DataFrame(dict(age=[5, 6, np.nan],
                       born=[pd.NaT, pd.Timestamp('1939-05-27'),
                             pd.Timestamp('1940-04-25')],
                       name=['Alfred', 'Batman', ''],
                       toy=[None, 'Batmobile', 'Joker']))

de

,age,born,name,toy
0,5.0,NaT,Alfred,None
1,6.0,1939-05-27,Batman,Batmobile
2,NaN,1940-04-25,,Joker


In [49]:
de.isna()

,age,born,name,toy
0,False,True,False,True
1,False,False,False,False
2,True,False,False,False


In [81]:
de.isnull()

,age,born,name,toy
0,False,True,False,True
1,False,False,False,False
2,True,False,False,False


In [83]:
de['age'].isna()

0    False
1    False
2     True
Name: age, dtype: bool

Nos cuenta cuantos nulos existen en cada columna del dataframe

In [53]:
de.isna().sum()

age     1
born    1
name    0
toy     1
dtype: int64

##### Ejemplo 2

In [3]:
path_normal = "paises.csv"
# Podemos usar para el formato de letras -> encoding='latin-1'
# Podemos usar para no utilizar la columna 'Unnamed' como index -> df = pd.read_csv(path_normal, encoding= 'iso-8859-1', index_col=0)
df = pd.read_csv(path_normal, encoding= 'iso-8859-1').drop(columns = 'Unnamed: 0')
df.shape

(194, 13)

In [4]:
df.isna().sum()

country       0
region        0
gdp          15
school        6
adfert        0
chldmort      1
life          0
pop           0
urban         0
femlab       17
literacy     59
co2           9
gini        113
dtype: int64

In [5]:
for i in df.region.unique():
    print(i)
    print(df.loc[df.region==i].shape)

Africa
(52, 13)
Americas
(35, 13)
Asia
(49, 13)
Europe
(43, 13)
Oceania
(15, 13)


In [6]:
for i in df.region.unique():
    print(i)
    print(df.loc[df.region==i].shape)
    na = df.loc[df.region==i].isna().sum()
    print(na)
    print('-'*50)

Africa
(52, 13)
country      0
region       0
gdp          2
school       1
adfert       0
chldmort     0
life         0
pop          0
urban        0
femlab       1
literacy     5
co2          1
gini        30
dtype: int64
--------------------------------------------------
Americas
(35, 13)
country      0
region       0
gdp          2
school       0
adfert       0
chldmort     0
life         0
pop          0
urban        0
femlab       4
literacy    15
co2          0
gini        17
dtype: int64
--------------------------------------------------
Asia
(49, 13)
country      0
region       0
gdp          3
school       1
adfert       0
chldmort     1
life         0
pop          0
urban        0
femlab       0
literacy     8
co2          0
gini        24
dtype: int64
--------------------------------------------------
Europe
(43, 13)
country      0
region       0
gdp          4
school       2
adfert       0
chldmort     0
life         0
pop          0
urban        0
femlab       6
literacy 

In [7]:
for i in df.region.unique():
    print(i)
    print(df.loc[df.region==i].shape)
    na = df.loc[df.region==i].isna().sum()[df.loc[df.region==i].isna().sum()!=0]
    print(na)
    print('-'*50)

Africa
(52, 13)
gdp          2
school       1
femlab       1
literacy     5
co2          1
gini        30
dtype: int64
--------------------------------------------------
Americas
(35, 13)
gdp          2
femlab       4
literacy    15
gini        17
dtype: int64
--------------------------------------------------
Asia
(49, 13)
gdp          3
school       1
chldmort     1
literacy     8
gini        24
dtype: int64
--------------------------------------------------
Europe
(43, 13)
gdp          4
school       2
femlab       6
literacy    21
co2          6
gini        27
dtype: int64
--------------------------------------------------
Oceania
(15, 13)
gdp          4
school       2
femlab       6
literacy    10
co2          2
gini        15
dtype: int64
--------------------------------------------------


#### **`>` notna( )**

- La inversa de **`isna()`**

##### Ejemplo 1

In [82]:
de.notna()

,age,born,name,toy
0,True,False,True,False
1,True,True,True,True
2,False,True,True,True


In [84]:
de['age'].notna()


0     True
1     True
2    False
Name: age, dtype: bool

Suma la cantidad de valores no ausentes de cada columna del dataframe

In [85]:
de.notna().sum()

age     2
born    2
name    3
toy     2
dtype: int64

Suma la cantidad de valores no ausentes en la columna **`passengers`**

In [86]:
de['age'].notna().sum()

2

#### **`>` value_counts()**

- Mayor info de este método en el punto '**`obtener_cantidad_de_categorias_(frecuencia)`**'

In [3]:
dk = pd.DataFrame([[np.nan, 2, np.nan, 0],
                   [3, 4, np.nan, 1],
                   [np.nan, np.nan, np.nan, np.nan],
                   [np.nan, 3, np.nan, 4]],
                  columns=list("ABCD"))
dk

,A,B,C,D
0,NaN,2.0,NaN,0.0
1,3.0,4.0,NaN,1.0
2,NaN,NaN,NaN,NaN
3,NaN,3.0,NaN,4.0


Con dropna en False también podemos ver valores de índice NaN.

In [4]:
dk['A'].value_counts(dropna=False)

NaN    3
3.0    1
Name: A, dtype: int64

#### **`>` loc( )**

In [13]:
de = pd.DataFrame(dict(age=[5, 6, np.nan],
                       born=[pd.NaT, pd.Timestamp('1939-05-27'),
                             pd.Timestamp('1940-04-25')],
                       name=['Alfred', 'Batman', ''],
                       toy=[None, 'Batmobile', 'Joker']))

de

,age,born,name,toy
0,5.0,NaT,Alfred,None
1,6.0,1939-05-27,Batman,Batmobile
2,NaN,1940-04-25,,Joker


##### Ejemplo 1

Modificar un **`NaN`** 

In [14]:
de.loc[2, 'age'] = 20
de

,age,born,name,toy
0,5.0,NaT,Alfred,None
1,6.0,1939-05-27,Batman,Batmobile
2,20.0,1940-04-25,,Joker


#### **`>` dropna( )**

- Pandas es uno de los paquetes que facilita la importación y el análisis de datos. A veces los ficheros CSV tienen valores nulos, que luego se muestran como NaN en Pandas DataFrame. El método **`dropna()`** de Pandas permite al usuario analizar y eliminar Filas/Columnas con valores Nulos de diferentes maneras.

> Sintaxis: DataFrameName.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

**`Parámetros`**:

- **`axis`**: {**`0`** o '**`index`**', **`1`** o '**`columns`**'}, **`por defecto 0`**. Determina si se eliminan las filas o columnas que contienen valores omitidos.

    - **`0`**, o '**`index`**' : Elimina las filas que contienen valores perdidos.
    - **`1`**, o '**`columns`**' : Elimina las columnas que contienen valores omitidos.
    - Sólo se permite un único eje.
```
```
- **`how`**: how toma valores string de dos tipos solamente ('any' o 'all'). '**`any`**' elimina la fila/columna si CUALQUIER valor es nulo y '**`all`**' sólo si TODOS los valores son nulos. Por defecto ==> **`how = 'any'`**

- **`thresh`**: thresh toma un valor entero que indica la cantidad mínima de valores nulos a eliminar. 

- **`subset`**: Es un array que limita el proceso de eliminación a las filas/columnas pasadas por lista. 

- **`inplace`**: Es un booleano que realiza los cambios en el mismo dataframe si es True.

##### Ejemplo 1

In [87]:
da = pd.DataFrame([['Alfonso', 20, 'Antofagasta'], ['Javier', 30, ''], ['', None, '']],
                  columns=['nombre', 'edad', 'ciudad'])

da

,nombre,edad,ciudad
0,Alfonso,20.0,Antofagasta
1,Javier,30.0,
2,,NaN,


In [88]:
# Por defecto, dropna() lleva el parametro how = 'any' y axis=0
da.dropna()

,nombre,edad,ciudad
0,Alfonso,20.0,Antofagasta
1,Javier,30.0,


In [65]:
result = da.dropna()

result.loc[1,'ciudad'] = 'Santiago'

result

,nombre,edad,ciudad
0,Alfonso,20.0,Antofagasta
1,Javier,30.0,Santiago


##### Ejemplo 2

- Aquí estamos usando **`read_csv()`** para leer nuestro archivo CSV. Eliminación de filas con al menos 1 valor nulo. Se lee un dataframe y se eliminan todas las filas con valores nulos. Se compara el tamaño de los dataframes antiguos y nuevos para ver cuántas filas tenían al menos 1 valor Nulo.

In [91]:
data = pd.read_csv("nba.csv")
data

,Name,Team,Number,Position,Age,Height,Weight,College,Salary
0,Avery Bradley,Boston Celtics,0.0,PG,25.0,6-2,180.0,Texas,7730337.0
1,Jae Crowder,Boston Celtics,99.0,SF,25.0,6-6,235.0,Marquette,6796117.0
2,John Holland,Boston Celtics,30.0,SG,27.0,6-5,205.0,Boston University,NaN
3,R.J. Hunter,Boston Celtics,28.0,SG,22.0,6-5,185.0,Georgia State,1148640.0
4,Jonas Jerebko,Boston Celtics,8.0,PF,29.0,6-10,231.0,NaN,5000000.0
...,...,...,...,...,...,...,...,...,...
453,Shelvin Mack,Utah Jazz,8.0,PG,26.0,6-3,203.0,Butler,2433333.0
454,Raul Neto,Utah Jazz,25.0,PG,24.0,6-1,179.0,NaN,900000.0
455,Tibor Pleiss,Utah Jazz,21.0,C,26.0,7-3,256.0,NaN,2900000.0
456,Jeff Withey,Utah Jazz,24.0,C,26.0,7-0,231.0,Kansas,947276.0


In [98]:
new_data = data.dropna(axis= 0, how='any')
new_data

,Name,Team,Number,Position,Age,Height,Weight,College,Salary
0,Avery Bradley,Boston Celtics,0.0,PG,25.0,6-2,180.0,Texas,7730337.0
1,Jae Crowder,Boston Celtics,99.0,SF,25.0,6-6,235.0,Marquette,6796117.0
3,R.J. Hunter,Boston Celtics,28.0,SG,22.0,6-5,185.0,Georgia State,1148640.0
6,Jordan Mickey,Boston Celtics,55.0,PF,21.0,6-8,235.0,LSU,1170960.0
7,Kelly Olynyk,Boston Celtics,41.0,C,25.0,7-0,238.0,Gonzaga,2165160.0
...,...,...,...,...,...,...,...,...,...
449,Rodney Hood,Utah Jazz,5.0,SG,23.0,6-8,206.0,Duke,1348440.0
451,Chris Johnson,Utah Jazz,23.0,SF,26.0,6-6,206.0,Dayton,981348.0
452,Trey Lyles,Utah Jazz,41.0,PF,20.0,6-10,234.0,Kentucky,2239800.0
453,Shelvin Mack,Utah Jazz,8.0,PG,26.0,6-3,203.0,Butler,2433333.0


In [101]:
print("Longitud del antiguo dataframe:", len(data),
      "\nLongitud del nuevo dataframe:", len(new_data), 
      "\nNúmero de filas con al menos 1 valor NA: ",
      (len(data)-len(new_data)))

Longitud del antiguo dataframe: 458 
Longitud del nuevo dataframe: 364 
Número de filas con al menos 1 valor NA:  94


##### Ejemplo 3

- Cambio de **`axis`** y uso de parametros **`how`** e **`inplace`**. Se crean dos Dataframes. Se añade una columna con todos los valores = none al nuevo Dataframe. Se verifican los nombres de las columnas para ver si la columna Null se insertó correctamente. Luego se compara el número de columnas antes y después de eliminar los valores NaN.

In [106]:
data = pd.read_csv("nba.csv")
data

,Name,Team,Number,Position,Age,Height,Weight,College,Salary
0,Avery Bradley,Boston Celtics,0.0,PG,25.0,6-2,180.0,Texas,7730337.0
1,Jae Crowder,Boston Celtics,99.0,SF,25.0,6-6,235.0,Marquette,6796117.0
2,John Holland,Boston Celtics,30.0,SG,27.0,6-5,205.0,Boston University,NaN
3,R.J. Hunter,Boston Celtics,28.0,SG,22.0,6-5,185.0,Georgia State,1148640.0
4,Jonas Jerebko,Boston Celtics,8.0,PF,29.0,6-10,231.0,NaN,5000000.0
...,...,...,...,...,...,...,...,...,...
453,Shelvin Mack,Utah Jazz,8.0,PG,26.0,6-3,203.0,Butler,2433333.0
454,Raul Neto,Utah Jazz,25.0,PG,24.0,6-1,179.0,NaN,900000.0
455,Tibor Pleiss,Utah Jazz,21.0,C,26.0,7-3,256.0,NaN,2900000.0
456,Jeff Withey,Utah Jazz,24.0,C,26.0,7-0,231.0,Kansas,947276.0


In [105]:
new = pd.read_csv("nba.csv")

new["Null Column"] = None

new


,Name,Team,Number,Position,Age,Height,Weight,College,Salary,Null Column
0,Avery Bradley,Boston Celtics,0.0,PG,25.0,6-2,180.0,Texas,7730337.0,None
1,Jae Crowder,Boston Celtics,99.0,SF,25.0,6-6,235.0,Marquette,6796117.0,None
2,John Holland,Boston Celtics,30.0,SG,27.0,6-5,205.0,Boston University,NaN,None
3,R.J. Hunter,Boston Celtics,28.0,SG,22.0,6-5,185.0,Georgia State,1148640.0,None
4,Jonas Jerebko,Boston Celtics,8.0,PF,29.0,6-10,231.0,NaN,5000000.0,None
...,...,...,...,...,...,...,...,...,...,...
453,Shelvin Mack,Utah Jazz,8.0,PG,26.0,6-3,203.0,Butler,2433333.0,None
454,Raul Neto,Utah Jazz,25.0,PG,24.0,6-1,179.0,NaN,900000.0,None
455,Tibor Pleiss,Utah Jazz,21.0,C,26.0,7-3,256.0,NaN,2900000.0,None
456,Jeff Withey,Utah Jazz,24.0,C,26.0,7-0,231.0,Kansas,947276.0,None


In [109]:
print(data.columns.values, "\n", new.columns.values)

['Name' 'Team' 'Number' 'Position' 'Age' 'Height' 'Weight' 'College'
 'Salary'] 
 ['Name' 'Team' 'Number' 'Position' 'Age' 'Height' 'Weight' 'College'
 'Salary' 'Null Column']


In [116]:
print("Número de columnas del Dataframe 'data':", len(data.dtypes),
    "\nNúmero de columnas antes de eliminar la columna Null del Dataframe 'new':", len(new.dtypes))

Número de columnas del Dataframe 'data': 9 
Número de columnas antes de eliminar la columna Null del Dataframe 'new': 10


In [118]:
# 'inplace' mantiene la modificacion en el Dataframe, sin la necesidad de crear un nuevo Dataframe
new.dropna(axis=1, how='all', inplace=True)
new

,Name,Team,Number,Position,Age,Height,Weight,College,Salary
0,Avery Bradley,Boston Celtics,0.0,PG,25.0,6-2,180.0,Texas,7730337.0
1,Jae Crowder,Boston Celtics,99.0,SF,25.0,6-6,235.0,Marquette,6796117.0
2,John Holland,Boston Celtics,30.0,SG,27.0,6-5,205.0,Boston University,NaN
3,R.J. Hunter,Boston Celtics,28.0,SG,22.0,6-5,185.0,Georgia State,1148640.0
4,Jonas Jerebko,Boston Celtics,8.0,PF,29.0,6-10,231.0,NaN,5000000.0
...,...,...,...,...,...,...,...,...,...
453,Shelvin Mack,Utah Jazz,8.0,PG,26.0,6-3,203.0,Butler,2433333.0
454,Raul Neto,Utah Jazz,25.0,PG,24.0,6-1,179.0,NaN,900000.0
455,Tibor Pleiss,Utah Jazz,21.0,C,26.0,7-3,256.0,NaN,2900000.0
456,Jeff Withey,Utah Jazz,24.0,C,26.0,7-0,231.0,Kansas,947276.0


In [120]:
print("Número de columnas del Dataframe 'data':", len(data.dtypes),
    "\nNúmero de columnas despues de eliminar la columna Null del Dataframe 'new':", len(new.dtypes))

Número de columnas del Dataframe 'data': 9 
Número de columnas despues de eliminar la columna Null del Dataframe 'new': 9


##### Ejemplo 4

In [2]:
df = pd.read_csv("qog_std_cs_jan18.csv")
df.head(10)

,ccode,cname,ccodealp,ccodecow,ccodewb,version,aid_cpnc,aid_cpsc,aid_crnc,aid_crnio,...,wvs_relacc,wvs_relsch,wvs_relsci,wvs_satfin,wvs_satlif,wvs_screl,wvs_subh,wvs_survself,wvs_tradrat,wvs_trust
0,4,Afghanistan,AFG,700.0,4.0,QoGStdCSJan18,NaN,NaN,29.0,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8,Albania,ALB,339.0,8.0,QoGStdCSJan18,NaN,NaN,26.0,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12,Algeria,DZA,615.0,12.0,QoGStdCSJan18,NaN,NaN,21.0,6.0,...,3.710616,1.704692,3.681347,5.988125,6.301029,5.339371,3.805579,-0.409807,-0.498164,0.179286
3,20,Andorra,AND,232.0,20.0,QoGStdCSJan18,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,24,Angola,AGO,540.0,24.0,QoGStdCSJan18,NaN,NaN,22.0,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,28,Antigua and Barbuda,ATG,58.0,28.0,QoGStdCSJan18,NaN,NaN,11.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,31,Azerbaijan,AZE,373.0,31.0,QoGStdCSJan18,NaN,NaN,25.0,15.0,...,2.922372,2.025111,2.305843,5.581910,6.662893,4.948389,3.753472,-0.667609,-0.416085,0.152803
7,32,Argentina,ARG,160.0,32.0,QoGStdCSJan18,NaN,NaN,23.0,7.0,...,1.850180,2.163982,2.091454,6.517899,7.488213,6.504242,3.989496,0.161507,-0.103415,0.198548
8,36,Australia,AUS,900.0,36.0,QoGStdCSJan18,134.0,2.866382e+09,NaN,NaN,...,1.606337,2.209082,1.755517,6.321970,7.203336,4.063055,4.069681,1.044364,0.422270,0.518140
9,40,Austria,AUT,305.0,40.0,QoGStdCSJan18,103.0,3.775100e+08,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.loc[df['undp_hdi'].isnull()]

,ccode,cname,ccodealp,ccodecow,ccodewb,version,aid_cpnc,aid_cpsc,aid_crnc,aid_crnio,...,wvs_relacc,wvs_relsch,wvs_relsci,wvs_satfin,wvs_satlif,wvs_screl,wvs_subh,wvs_survself,wvs_tradrat,wvs_trust
37,158,Taiwan,TWN,713.0,158.0,QoGStdCSJan18,28.0,32709472.0,NaN,NaN,...,1.950127,2.490061,2.072234,6.391475,6.896482,5.671831,4.112848,-0.18797,0.759673,0.30888
89,408,"Korea, North",PRK,731.0,408.0,QoGStdCSJan18,NaN,NaN,18.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,492,Monaco,MCO,221.0,492.0,QoGStdCSJan18,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119,520,Nauru,NRU,970.0,520.0,QoGStdCSJan18,NaN,NaN,4.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
129,584,Marshall Islands,MHL,983.0,584.0,QoGStdCSJan18,NaN,NaN,9.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
148,674,San Marino,SMR,331.0,674.0,QoGStdCSJan18,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
159,706,Somalia,SOM,520.0,706.0,QoGStdCSJan18,NaN,NaN,25.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
179,798,Tuvalu,TUV,947.0,798.0,QoGStdCSJan18,NaN,NaN,8.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Antes de eliminar registros, podemos revisar que registros poseen nulos en la columna **`undp_hdi`**

In [4]:
df.loc[df['undp_hdi'].isnull()].cname.unique()

array(['Taiwan', 'Korea, North', 'Monaco', 'Nauru', 'Marshall Islands',
       'San Marino', 'Somalia', 'Tuvalu'], dtype=object)

In [5]:
df.loc[df['undp_hdi'].isnull()].cname.to_list()

['Taiwan',
 'Korea, North',
 'Monaco',
 'Nauru',
 'Marshall Islands',
 'San Marino',
 'Somalia',
 'Tuvalu']

Elimino las filas que tienen nulos en la columna **`undp_hdi`**

In [6]:
df.dropna(subset=['undp_hdi'])

,ccode,cname,ccodealp,ccodecow,ccodewb,version,aid_cpnc,aid_cpsc,aid_crnc,aid_crnio,...,wvs_relacc,wvs_relsch,wvs_relsci,wvs_satfin,wvs_satlif,wvs_screl,wvs_subh,wvs_survself,wvs_tradrat,wvs_trust
0,4,Afghanistan,AFG,700.0,4.0,QoGStdCSJan18,NaN,NaN,29.0,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8,Albania,ALB,339.0,8.0,QoGStdCSJan18,NaN,NaN,26.0,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12,Algeria,DZA,615.0,12.0,QoGStdCSJan18,NaN,NaN,21.0,6.0,...,3.710616,1.704692,3.681347,5.988125,6.301029,5.339371,3.805579,-0.409807,-0.498164,0.179286
3,20,Andorra,AND,232.0,20.0,QoGStdCSJan18,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,24,Angola,AGO,540.0,24.0,QoGStdCSJan18,NaN,NaN,22.0,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,860,Uzbekistan,UZB,704.0,860.0,QoGStdCSJan18,NaN,NaN,21.0,16.0,...,3.247667,2.191377,2.171875,4.076407,7.888145,5.364856,3.950634,NaN,NaN,0.140921
190,862,Venezuela,VEN,101.0,862.0,QoGStdCSJan18,NaN,NaN,20.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
191,882,Samoa,WSM,990.0,882.0,QoGStdCSJan18,NaN,NaN,10.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
192,887,Yemen,YEM,679.0,887.0,QoGStdCSJan18,NaN,NaN,26.0,16.0,...,3.598178,1.697442,3.752032,4.671357,5.887310,3.384707,3.892000,-0.591479,-0.875164,0.403987


Podemos utilizarlo en series 

In [10]:
df['undp_hdi'].dropna()

0      0.479
1      0.762
2      0.743
3      0.857
4      0.531
       ...  
189    0.697
190    0.769
191    0.702
192    0.499
193    0.576
Name: undp_hdi, Length: 186, dtype: float64

In [11]:
df.dropna(subset=['undp_hdi'])['undp_hdi']

0      0.479
1      0.762
2      0.743
3      0.857
4      0.531
       ...  
189    0.697
190    0.769
191    0.702
192    0.499
193    0.576
Name: undp_hdi, Length: 186, dtype: float64

Comparo las filas que tenia en un principio con las que acabo de eliminar

In [7]:
df.shape, df.dropna(subset=['undp_hdi']).shape

((194, 1882), (186, 1882))

#### **`>` fillna( )**

- El método **`fillna()`** sustituye los valores NULL por un valor especificado.

- El método **`fillna()`** devuelve un nuevo objeto DataFrame a menos que el parámetro inplace se establezca en True, en cuyo caso el método fillna() realiza la sustitución en el DataFrame original en su lugar.

> Sintaxis: DataFrameName.fillna(value, method, axis, inplace, limit, downcast)

**`Parámetros`**:

- **`value`**: {Number, String, Dictionary, Series, DataFrame}. Obligatorio, Especifica el valor por el que se sustituirán los valores NULL. También pueden ser valores para toda la fila o columna.

- **`method`**: {'backfill', 'bfill', 'pad', 'ffill', None}. Opcional, por defecto "None". Especifica el método que se utilizará al sustituir.

- **`axis`**: {**`0`** o '**`index`**', **`1`** o '**`columns`**'}. Opcional, por defecto 0. El eje para rellenar los valores NULL a lo largo.

- **`inplace`**: Opcional, por defecto **`False`**. Si es True: el reemplazo se realiza en el DataFrame actual. Si es False: devuelve una copia donde se realiza el reemplazo.

- **`limit`**: {Number, None}. Opcional, por defecto None. Especifica el número máximo de valores NULL a rellenar (si se especifica el método).

##### Ejemplo 1

In [121]:
dk = pd.DataFrame([[np.nan, 2, np.nan, 0],
                   [3, 4, np.nan, 1],
                   [np.nan, np.nan, np.nan, np.nan],
                   [np.nan, 3, np.nan, 4]],
                  columns=list("ABCD"))
dk

,A,B,C,D
0,NaN,2.0,NaN,0.0
1,3.0,4.0,NaN,1.0
2,NaN,NaN,NaN,NaN
3,NaN,3.0,NaN,4.0


In [122]:
# Reemplazo TODOS LOS VALORES NaN con un 0
dk.fillna(0)

,A,B,C,D
0,0.0,2.0,0.0,0.0
1,3.0,4.0,0.0,1.0
2,0.0,0.0,0.0,0.0
3,0.0,3.0,0.0,4.0


In [124]:
# Sustituye todos los elementos NaN de las columnas 'A', 'B', 'C', y 'D', por 0, 1, 2, y 3 respectivamente.
values = {"A": 0, "B": 1, "C": 2, "D": 3}
dk.fillna(value=values)

,A,B,C,D
0,0.0,2.0,2.0,0.0
1,3.0,4.0,2.0,1.0
2,0.0,1.0,2.0,3.0
3,0.0,3.0,2.0,4.0


In [125]:
# Sólo sustituye el primer elemento NaN
values = {"A": 0, "B": 1, "C": 2, "D": 3}
dk.fillna(value=values, limit=1)

,A,B,C,D
0,0.0,2.0,2.0,0.0
1,3.0,4.0,NaN,1.0
2,NaN,1.0,NaN,3.0
3,NaN,3.0,NaN,4.0


In [129]:
# Cuando se rellena utilizando un DataFrame, el reemplazo se produce a lo largo de los mismos nombres de columna y los mismos índices
dk2 = pd.DataFrame(np.zeros((4, 4)), columns=list("ABCE"))
dk2

,A,B,C,E
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0


In [131]:
dk

,A,B,C,D
0,NaN,2.0,NaN,0.0
1,3.0,4.0,NaN,1.0
2,NaN,NaN,NaN,NaN
3,NaN,3.0,NaN,4.0


In [130]:
dk.fillna(dk2)

,A,B,C,D
0,0.0,2.0,0.0,0.0
1,3.0,4.0,0.0,1.0
2,0.0,0.0,0.0,NaN
3,0.0,3.0,0.0,4.0
